# P3W6: Herhaling

In [41]:
# import
import pandas as pd 
from scipy.stats import binom, norm, poisson, t, ttest_1samp,chisquare
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import association_rules
import math
import numpy as np



## Vraag 1:
Uit een dataset van het vorige jaar zijn volgende kansen gekend. 79% van de studenten
nemen voor het eerst deel aan het examen Data Science. Respectievelijk 12.65%, 4.30% en
4.05% van de studenten nemen voor de 2de, 3de en 4de keer deel aan het examen.
Als de student voor het eerst deelneemt dan is de kans dat hij geslaagd is 60%. Bij de
tweede, derde en vierde deelname is de kans dat hij geslaagd is respectievelijk 40%, 12%
en 2%.
a) Er is 53,057% kans dat een willekeurige gekozen student van vorig jaar geslaagd is.
Noteer de berekening waarmee je op basis van de gegevens uit de opgave aan dit
percentage komt.
b) 24% van de studenten die vorig jaar deelnamen aan het examen Data Science zijn
gedoopt. Het percentage geslaagde studenten van de groep gedoopte studenten is
exact hetzelfde als het percentage geslaagde studenten voor de volledige groep. Stel
dat de docent uit de volledige groep studenten (die vorig jaar deelnamen aan het
examen) één student kiest. Wat is de kans dat deze student gedoopt werd en vorig
jaar geslaagd was voor Data Science.
c) De docent roept 15 studenten naar voor die vorig jaar voor de eerste keer deelnamen
aan het examen. Wat is de kans dat daarvan 10 of meer studenten geslaagd zijn

In [42]:
# a)
kans = 60/100 *0.79 +40/100 * 0.1265 + 12/100 * 0.0430 + 2/100 * 0.0405
print(kans)
# b)
kansgedoopt = kans * 0.24
print(kansgedoopt)
# c)
print((1 - binom.cdf(9,15,0.6))*100) 

0.53057
0.1273368
40.3215550414848


## Extra vraag
Een theoretisch rijexamen bevat 50 meerkeuze vragen. Je dient 43/50 te halen om te slage. Stel dat elke meerkeuze vraag 4 mogelijke antwoorden heeft. 

In [43]:
# a) Wat is de kans om te kunnen slagen (43/50) wanneer je enkel gokt (4 mogelijkheden)?
slaagnummer = 43/50
kansOpJuist = 1 / 4
test =  1- binom.cdf(42,50,kansOpJuist)
test

0.0

In [44]:
# b) Wat is de kans om te slagen indien je van 35 vragen zeker bent (en dus 15 keer hebt gegokt)?
n = 8
aantal = 43-50-n-1 
kansOpJuist = 1 / 4
test = 1 - binom.cdf(aantal,n,kansOpJuist)
test

1.0

In [45]:
# c) Wat is de kans om te kunnen slagen (43/50) wanneer je enkel gokt en er maar 2 mogelijkheden per vraag zijn?
kansOpJuist= 1 / 2
slaagnummer = 43-1
n = 50
test =  1- binom.cdf(slaagnummer,n,kansOpJuist)
test

1.049338713698944e-07

## Vraag 2
Vraag 2:
De gemiddelde studietijd per opleidingsonderdeel bedraagt 4.5 uur per week in de opleiding.
We willen nagaan of de studietijd van de opleiding significant verschilt van het gemiddelde
voor het vak Data Science (op basis van een steekproef). Gebruik hiervoor het inputbestand
a) Welke alternatieve hypothese kunnen we gebruiken?
b) Wat is het aanvaardingsinterval voor de studietijd van de opleiding bij =0.05? Schrijf
ook op hoe je tot dit getal gekomen bent.
c) Welke waarde voor de factor heb je gebruikt om het aanvaardingsinterval te
berekenen?
d) Wat kan je nu zeggen over de nulhypothese?
e) Bereken de p-waarde die je kan gebruiken om deze bewering te staven (bij =)
f) Formuleer een conclusie op basis van de p-waarde. Vul daarvoor de volgende zin
aan (gebruik in ieder geval de bekomen p-waarde in je conclusie):
Als de gemiddelde studieduur van de populatie gelijk is aan 4,5 UUR dan is

In [46]:
data = pd.read_csv('../DataSets/studyResult.csv',delimiter=";", decimal=",")
mu0 = 4.5
alpha = 0.05
p = 1-alpha
s= data['study time'].std()
x_bar = data['study time'].mean()
n=len(data['study time'])
print(t.interval(confidence=p,df=n-1,loc=mu0,scale=s/math.sqrt(n)))

print(x_bar)
# niet accepteren kijken naar xbar

(4.301534060067161, 4.698465939932839)
4.075189873417722


In [47]:
t.ppf((1-alpha/2),df=n-1)
ttest_1samp(data['study time'],mu0)

TtestResult(statistic=-4.2081682585373015, pvalue=3.19125975826884e-05, df=394)

## Vraag 3
Vraag 3:
We willen weten of het uitgaansgedrag van
onze studenten verschilt met die van de
hogeschool
Uit bevraging van alle studenten van de
hogeschool blijkt volgende verdeling:
a) Bereken ²
b) Wat is de p waarde voor deze verdeling?
c) Ik wil een besluit nemen met een
betrouwbaarheid van 90%. Welk besluit
neem ik?

In [66]:
data = pd.read_csv('../DataSets/studyResult.csv',delimiter=";", decimal=",")
measured = data['partying'].value_counts()
n =measured.sum()
expected = np.array([0.35,0.3,0.2,0.1,0.05]) * n
chisquare(measured,expected)

Power_divergenceResult(statistic=8.288728149487643, pvalue=0.08155624992038794)

## Vraag 4
Een winkel heeft een aantal winkelkarren geanalyseerd. Ga op zoek naar de assocatieregels.
Lees het groceries.cvs bestand in. Neem min_support=0.05 en min_confidence=0.1. Zoek de regel yogurt → whole milk op

In [49]:
data = pd.read_csv("../DataSets/groceries.csv")

 # Machine Learning Library Extentions


def get_item_list (string):
    items = string [1:-1]
    return items.split(';')
transactions = data['items'].apply(get_item_list).to_numpy()
te = TransactionEncoder()
dataset = te.fit(transactions).transform(transactions)
transactionstable = pd.DataFrame(dataset,
columns=te.columns_, index= data.transaction)
transactionstable
itemsets = fpgrowth(transactionstable,min_support=0.05 , use_colnames=True)
rules = association_rules(itemsets, metric='confidence',min_threshold=0.1)
rules.sort_values('confidence',ascending=False)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
1,(yogurt),(whole milk),0.139502,0.255516,0.056024,0.401603,1.571735,0.020379,1.244132,0.422732
3,(other vegetables),(whole milk),0.193493,0.255516,0.074835,0.386758,1.513634,0.025394,1.214013,0.420750
5,(rolls/buns),(whole milk),0.183935,0.255516,0.056634,0.307905,1.205032,0.009636,1.075696,0.208496
2,(whole milk),(other vegetables),0.255516,0.193493,0.074835,0.292877,1.513634,0.025394,1.140548,0.455803
4,(whole milk),(rolls/buns),0.255516,0.183935,0.056634,0.221647,1.205032,0.009636,1.048452,0.228543
0,(whole milk),(yogurt),0.255516,0.139502,0.056024,0.219260,1.571735,0.020379,1.102157,0.488608
